In [ ]:
import pandas as pd

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, roc_auc_score)
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
print('imports done')

# Use the direct path to the file in the environment
input_file = 'sp1500_company_info.csv'
df_raw = pd.read_csv(input_file)

# find the highest missing percent columns
missing_pct = df_raw.isnull().mean().sort_values(ascending=False)
print('Missing percentages before cleaning')
print(missing_pct.head(10))

# print out the free cashflow range, median
print('Free Cashflow metrics')
cash_flow_nulls = df_raw['freeCashflow'].isnull().sum()
print('Free Cashflow count of nulls:', cash_flow_nulls)
cashflow_range = df_raw['freeCashflow'].max() - df_raw['freeCashflow'].min()
print('free Cashflow range:', cashflow_range)
cashflow_median = df_raw['freeCashflow'].median()
print('free Cashflow median:', cashflow_median)

# print out median value of returnOnEquity and OperatingMargins
print('ROE and margin medians')
print('ROE median:', df_raw['returnOnEquity'].median())
print('ROE unique values count', len(df_raw['returnOnEquity'].unique()))
print('Margin median', df_raw['operatingMargins'].median())
print('Margin unique values count', len(df_raw['operatingMargins'].unique()))

cell started
imports done
Missing percentages before cleaning
tickerChangeDate    0.999336
prevTicker          0.999336
industrySymbol      0.999336
ipoExpectedDate     0.961462
nameChangeDate      0.897674
prevName            0.897674
fax                 0.798671
address2            0.586047
trailingPegRatio    0.581395
irWebsite           0.482392
dtype: float64
Free Cashflow metrics
Free Cashflow count of nulls: 243
free Cashflow range: 94629746688.0
free Cashflow median: 276585936.0
ROE and margin medians
ROE median: 0.11316
ROE unique values count 1385
Margin median 0.15403
Margin unique values count 1473


First prompt is done after data is downloaded with existing script. The prompt uploads all code already written and an ai_rules.txt containing a description of the task and rules to use for the duration of the conversation. Prompt snippet: "Given the ai_rules.txt document and the following code I have already written, what are some ways I can test the validity of my model? There are 5 different classifications that can be selected by the decision tree and I would like to compute the overall accuracy and auc_roc score of this tree. Is there a built in model performance library I can use?"

Part 3
I/O Cases

In [ ]:
def clean_data(df):
    df_clean = df.copy()
    # keep only rows with ROE values
    df_clean.dropna(subset=['returnOnEquity'], inplace=True)
    # split ROE and operating margins to quartiles, then get composite score
    df_clean['label'] = (
            (pd.qcut(df_clean['returnOnEquity'], q=4, labels=False)) +
            (pd.qcut(df_clean['operatingMargins'], q=4, labels=False))
    )
    # remove columns that have no value for analysis, have large amount of
    # missing data

    drop_cols = ['companyOfficers', 'website', 'phone', 'irWebsite',
                 'longBusinessSummary', 'address1', 'tradeable', 'quoteType',
                 'symbol', 'language', 'region', 'quoteSourceName',
                 'triggerable', 'customPriceAlertConfidence', 'marketState',
                 'exchangeDataDelayedBy', 'sourceInterval', 'cryptoTradeable',
                 'shortName', 'longName', 'hasPrePostMarketData',
                 'corporateActions', 'messageBoardId', 'exchangeTimezoneName',
                 'exchangeTimezoneShortName', 'gmtOffSetMilliseconds', 'fax',
                 'market', 'esgPopulated', 'address2', 'displayName',
                 'ipoExpectedDate', 'prevName', 'nameChangeDate',
                 'industrySymbol', 'prevTicker', 'tickerChangeDate',
                 'trailingPegRatio', 'lastSplitDate', 'returnOnEquity',
                 'operatingMargins', 'lastSplitFactor']

    df_clean.drop(drop_cols, inplace=True, axis=1)
    return df_clean

df_cleaned = clean_data(df_raw)

# get missing percentages after cleaning
missing_pct = df_cleaned.isnull().mean().sort_values(ascending=False)
print('Missing percentages after cleaning')
print(missing_pct.head(10))

# show all remaining columns that are used for analysis
print(df_cleaned.columns.tolist())
print('Number of columns in df_cleaned:', df_cleaned.shape[1])

# show median for the new label column and the value counts
print('label median and value counts')
print(df_cleaned['label'].value_counts())
print('label median:', df_cleaned['label'].median())

Missing percentages after cleaning
fiveYearAvgDividendYield    0.345583
dividendRate                0.321555
dividendYield               0.321555
dividendDate                0.269965
lastDividendValue           0.258657
lastDividendDate            0.258657
exDividendDate              0.257951
earningsGrowth              0.196466
earningsQuarterlyGrowth     0.192933
recommendationMean          0.154770
dtype: float64
['city', 'state', 'zip', 'country', 'industry', 'industryKey', 'industryDisp', 'sector', 'sectorKey', 'sectorDisp', 'fullTimeEmployees', 'auditRisk', 'boardRisk', 'compensationRisk', 'shareHolderRightsRisk', 'overallRisk', 'governanceEpochDate', 'compensationAsOfEpochDate', 'executiveTeam', 'maxAge', 'priceHint', 'previousClose', 'open', 'dayLow', 'dayHigh', 'regularMarketPreviousClose', 'regularMarketOpen', 'regularMarketDayLow', 'regularMarketDayHigh', 'dividendRate', 'dividendYield', 'exDividendDate', 'payoutRatio', 'fiveYearAvgDividendYield', 'beta', 'trailingPE', 'forw

These results show the missing percentages are decreased after removing features with high missing percentages as well as the changes to make the label categorical feature

In [ ]:
# get all features, exclude label
features = df_cleaned.drop(columns=['label'])
target = df_cleaned['label']

X_train, X_test, y_train, y_test = train_test_split(features,
                                                    target,
                                                    test_size=0.2,
                                                    random_state=44,
                                                    stratify=target)

# use median imputing for numerical values after test train split
# impute X train and X test
# get the numerical and categorical columns
# AI prompting was done to get this pipeline but the code was significantly
# refactored to simplify the process
num_features = features.select_dtypes(include=['int64', 'float64']).columns
cat_features = features.select_dtypes(include=['object']).columns

# define a pipeline for numerical imputing and min max scaling
num_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())])  # scale numerical features so no neg values


# define categorical feature pipeline with imputing and ordinal encoding
cat_pipeline_oe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(handle_unknown="use_encoded_value",
                               unknown_value=-1))
])

# add transformers to the columnTransformer
preprocessor_oe = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, num_features),
        ('cat', cat_pipeline_oe, cat_features)
    ],
    remainder='drop'  # ensures all columns are handled or dropped
)

# This section is to show the effects of the training for freeCashflow
# Fit the preprocessor on the training data
# AI generated code chunk - it is quite efficient and well commented
preprocessor_oe.fit(X_train)

# Transform both train and test sets
X_train_processed = preprocessor_oe.transform(X_train)
X_test_processed = preprocessor_oe.transform(X_test)

# Get the feature names (for numeric + encoded categorical features)
feature_names = preprocessor_oe.get_feature_names_out()

# Convert transformed data back into a DataFrame
X_train_df = pd.DataFrame(X_train_processed, columns=feature_names, index=X_train.index)
X_test_df = pd.DataFrame(X_test_processed, columns=feature_names, index=X_test.index)

# Concatenate them to get a full processed dataset for analysis
X_full_processed = pd.concat([X_train_df, X_test_df])

# Find the exact transformed column name
[col for col in X_full_processed.columns if "freeCashflow" in col]

fc_col = 'num__freeCashflow'

# Compute post-imputation + scaling stats and print results
print(f"Median of {fc_col}:", X_full_processed[fc_col].median())
print(f"Number of nulls in {fc_col}:", X_full_processed[fc_col].isnull().sum())
print(f"Range of {fc_col}: {X_full_processed[fc_col].max() - X_full_processed[fc_col].min()}")

Median of num__freeCashflow: 0.15759575485446004
Number of nulls in num__freeCashflow: 0
Range of num__freeCashflow: 1.502673635228764


Here we can see that the min max scaler is working as expected and that the range of free cash flow might greater than 1 since the scaler is fit on the train data, then applied to all data.

In [ ]:
# fit ordinal encoded model
model_dt_oe = Pipeline(steps=[('preprocessor', preprocessor_oe),
                           ('classifier',
                            DecisionTreeClassifier(max_depth=6,
                                                   min_samples_leaf=8,
                                                   min_samples_split=2,
                                                   random_state=44,
                                                   criterion='gini'))])

# model fit with no grid search
model_dt_oe.fit(X_train, y_train) # fit model
y_pred_oe = model_dt_oe.predict(X_test) # get the prediction
y_pred_proba_oe = model_dt_oe.predict_proba(X_test) # get pred probability
dt_acc_oe = accuracy_score(y_test, y_pred_oe)

# AI generated - quite efficient and easily understood
auc_oe = roc_auc_score(y_test, y_pred_proba_oe, multi_class='ovr')
print("Decision Tree results label encoded")
print(f"Accuracy OE: {dt_acc_oe:.4f}")
print(classification_report(y_test, y_pred_oe))
print(confusion_matrix(y_test, y_pred_oe))
print(f"ROC-AUC OE: {auc_oe:.4f}")

Decision Tree results label encoded
Accuracy OE: 0.5053
              precision    recall  f1-score   support

           0       0.58      0.68      0.63        38
           1       0.37      0.26      0.30        27
           2       0.51      0.49      0.50        43
           3       0.42      0.42      0.42        52
           4       0.44      0.46      0.45        57
           5       0.57      0.64      0.60        45
           6       0.75      0.57      0.65        21

    accuracy                           0.51       283
   macro avg       0.52      0.50      0.51       283
weighted avg       0.50      0.51      0.50       283

[[26  7  4  1  0  0  0]
 [12  7  7  1  0  0  0]
 [ 6  4 21  9  3  0  0]
 [ 1  1  6 22 15  7  0]
 [ 0  0  2 15 26 12  2]
 [ 0  0  1  3 10 29  2]
 [ 0  0  0  1  5  3 12]]
ROC-AUC OE: 0.8243


In [ ]:
# define pipeline for one hot encoding and train model

# pipeline uses preexisting numerical steps, only differs for categorical

# define categorical feature pipeline with imputing and one hot encoding
cat_pipeline_ohe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore')),
])

# define processor for one hot encoding
preprocessor_ohe = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, num_features),
        ('cat', cat_pipeline_ohe, cat_features)  # cat will not have neg values
    ],
    remainder='drop'  # ensures all columns are handled or dropped
)

# one hot encoded model
model_dt_ohe = Pipeline(steps=[('preprocessor', preprocessor_ohe),
                           ('classifier',
                            DecisionTreeClassifier(max_depth=6,
                                                   min_samples_leaf=8,
                                                   min_samples_split=2,
                                                   random_state=99,
                                                   criterion='gini'))])

# model fit with one hot encoding
model_dt_ohe.fit(X_train, y_train) # fit model
y_pred_ohe = model_dt_ohe.predict(X_test) # get prediction
y_pred_proba_ohe = model_dt_ohe.predict_proba(X_test) # get prediction probs
dt_acc_ohe = accuracy_score(y_test, y_pred_ohe)

# AI generated
auc_ohe = roc_auc_score(y_test, y_pred_proba_ohe, multi_class='ovr')
print("Decision Tree results one-hot encoded")
print(f"Accuracy OHE: {dt_acc_ohe:.4f}")
print(classification_report(y_test, y_pred_ohe))
print(confusion_matrix(y_test, y_pred_ohe))
print(f"ROC-AUC OHE: {auc_ohe:.4f}")

Decision Tree results one-hot encoded
Accuracy OHE: 0.4982
              precision    recall  f1-score   support

           0       0.64      0.71      0.68        38
           1       0.45      0.37      0.41        27
           2       0.50      0.49      0.49        43
           3       0.37      0.37      0.37        52
           4       0.40      0.40      0.40        57
           5       0.57      0.64      0.60        45
           6       0.75      0.57      0.65        21

    accuracy                           0.50       283
   macro avg       0.53      0.51      0.51       283
weighted avg       0.50      0.50      0.50       283

[[27  6  4  1  0  0  0]
 [ 9 10  7  1  0  0  0]
 [ 5  5 21  9  3  0  0]
 [ 1  1  7 19 17  7  0]
 [ 0  0  2 18 23 12  2]
 [ 0  0  1  3 10 29  2]
 [ 0  0  0  1  5  3 12]]
ROC-AUC OHE: 0.8240


We can see there are very little changed results, this is likely because our numerical features likely contain stronger signals than the categorical features in our model.

In [ ]:
# selector added with OHE used

# chi2 selector is added to pick top 90 features
selector = SelectKBest(score_func=chi2, k=90)
model_dt_select = Pipeline(steps=[('preprocessor', preprocessor_ohe),
                                  ('selector', selector), # define selector here
                                  ('classifier',
                                  DecisionTreeClassifier(max_depth=6,
                                                         min_samples_leaf=8,
                                                         min_samples_split=2,
                                                         random_state=99,
                                                         criterion='gini'))])

# model fit with selector
model_dt_select.fit(X_train, y_train) # fit model with selector
y_pred_select = model_dt_select.predict(X_test) # get predictions
y_pred_proba_select = model_dt_select.predict_proba(X_test) # prediction probabilities
dt_acc_select = accuracy_score(y_test, y_pred_select)

# AI generated
auc = roc_auc_score(y_test, y_pred_proba_select, multi_class='ovr')
print("Decision Tree results with selector added and OHE processing")
print(f"Accuracy with selector and OHE processing: {dt_acc_select:.4f}")
print('Classification report with selector added and OHE processing')
print(classification_report(y_test, y_pred_select))
print('Confusion matrix with selector added and OHE processing')
print(confusion_matrix(y_test, y_pred_select))
print(f"ROC-AUC with selector and OHE processing: {auc:.4f}")

Decision Tree results with selector added and OHE processing
Accuracy with selector and OHE processing: 0.2367
Classification report with selector added and OHE processing
              precision    recall  f1-score   support

           0       0.34      0.37      0.35        38
           1       0.00      0.00      0.00        27
           2       0.16      0.16      0.16        43
           3       0.09      0.06      0.07        52
           4       0.24      0.65      0.35        57
           5       0.20      0.02      0.04        45
           6       0.83      0.24      0.37        21

    accuracy                           0.24       283
   macro avg       0.27      0.21      0.19       283
weighted avg       0.23      0.24      0.19       283

Confusion matrix with selector added and OHE processing
[[14  1 10  3 10  0  0]
 [ 6  0  9  3  9  0  0]
 [ 7  0  7  8 21  0  0]
 [ 6  0  6  3 36  1  0]
 [ 6  1  7  5 37  1  0]
 [ 1  1  3  9 29  1  1]
 [ 1  0  1  1 11  2  5]]
ROC-AU

Here we can see that adding a selector and using OHE results in dramatically worse accuracy and AUC-ROC because OHE explodes the number of features, which is then limited by our selector.

In [ ]:
# selector added with OE used

# use the exact same code except with a different processor
selector = SelectKBest(score_func=chi2, k=90)
model_dt_select = Pipeline(steps=[('preprocessor', preprocessor_oe),
                                  ('selector', selector),
                                  ('classifier',
                                  DecisionTreeClassifier(max_depth=6,
                                                         min_samples_leaf=8,
                                                         min_samples_split=2,
                                                         random_state=99,
                                                         criterion='gini'))])

# model fit with selector
model_dt_select.fit(X_train, y_train) # fit model
y_pred_select = model_dt_select.predict(X_test) # get predictions
y_pred_proba_select = model_dt_select.predict_proba(X_test) # get pred probs
dt_acc_select = accuracy_score(y_test, y_pred_select)

# AI generated
auc = roc_auc_score(y_test, y_pred_proba_select, multi_class='ovr')
print("Decision Tree results with selector added and OE processing")
print(f"Accuracy with selector and OHE processing: {dt_acc_select:.4f}")
print('Classification report with selector added and OE processing')
print(classification_report(y_test, y_pred_select))
print('Confusion matrix with selector added and OE processing')
print(confusion_matrix(y_test, y_pred_select))
print(f"ROC-AUC with selector and OE processing: {auc:.4f}")

Decision Tree results with selector added and OE processing
Accuracy with selector and OHE processing: 0.4947
Classification report with selector added and OE processing
              precision    recall  f1-score   support

           0       0.61      0.71      0.66        38
           1       0.33      0.30      0.31        27
           2       0.53      0.42      0.47        43
           3       0.38      0.40      0.39        52
           4       0.43      0.51      0.46        57
           5       0.59      0.58      0.58        45
           6       0.85      0.52      0.65        21

    accuracy                           0.49       283
   macro avg       0.53      0.49      0.50       283
weighted avg       0.51      0.49      0.50       283

Confusion matrix with selector added and OE processing
[[27  8  2  1  0  0  0]
 [10  8  8  1  0  0  0]
 [ 6  7 18  9  3  0  0]
 [ 1  1  5 21 20  4  0]
 [ 0  0  1 18 29  9  0]
 [ 0  0  0  4 13 26  2]
 [ 0  0  0  2  3  5 11]]
ROC-AUC w

Adding a selector and using ordinal encoding again reduces the accuracy and AUC-ROC but the impact is not as dramatic since ordinal encoding doesn't explode the number of features as OHE does.

In [ ]:
# now use a grid search with no selector and oe preprocessor to improve the
# performance before further analyzing the results
# define grid pipeline
grid_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor_oe),  # use the same preprocessor as before
    ('clf', DecisionTreeClassifier(random_state=44))  # use decision tree
])

# set the param grid that will be used in the pipeline
# initial params significantly changed from AI params
param_grid = {
    'clf__max_depth': [5, 6, 7, 8],  # max depth for tree
    'clf__min_samples_split': [2, 3, 4, 5],  # min samples split for tree
    'clf__min_samples_leaf': [6, 8, 10, 12, 13],  # min samples leaf for tree
    'clf__criterion': ['gini', 'entropy'] # split criterion
}

grid = GridSearchCV(
    grid_pipe,
    param_grid,
    cv=5, # cross validation with 5 folds
    scoring='roc_auc_ovr_weighted',  # non-binary classification scoring
    n_jobs=-1 # allows for all processors on machine to be used for faster search
)

grid.fit(X_train, y_train) # fit model
print("Best params from grid search:", grid.best_params_)
print("Best ROC-AUC from grid search:", grid.best_score_)

y_pred_grid = grid.predict(X_test) # get predictions
y_pred_proba_grid = grid.predict_proba(X_test) # get probabilities

Best params from grid search: {'clf__criterion': 'entropy', 'clf__max_depth': 5, 'clf__min_samples_leaf': 12, 'clf__min_samples_split': 2}
Best ROC-AUC from grid search: 0.8171709854630521


We can see the grid search AUC-ROC is slightly lower but the grid search tree is more generalized

In [10]:
# find misclassified label = 6 predictions
results_df = X_test.copy()
results_df['true_label'] = y_test
results_df['pred_label'] = y_pred_grid

# find rows where model predicted 6 but true label != 6

false_positives = results_df[(results_df['pred_label'] == 6) & (results_df['true_label'] != 6)]
print("False positives:")
print(false_positives)

False positives:
         city state    zip        country                       industry  \
720   Phoenix    AZ  85017  United States  Education & Training Services   
1392   Denver    CO  80203  United States                  Oil & Gas E&P   

                      industryKey                   industryDisp  \
720   education-training-services  Education & Training Services   
1392                  oil-gas-e-p                  Oil & Gas E&P   

                  sector           sectorKey          sectorDisp  ...  \
720   Consumer Defensive  consumer-defensive  Consumer Defensive  ...   
1392              Energy              energy              Energy  ...   

      fiftyTwoWeekHighChangePercent  fiftyTwoWeekChangePercent  dividendDate  \
720                       -0.155757                   37.12497           NaN   
1392                      -0.549978                  -48.97411  1.762128e+09   

      regularMarketChangePercent  postMarketTime  regularMarketTime  exchange  \
720    

We can see from this that only 2 companies were predicted to have a label 6, but did not have a true label of 6. This indicates that the model is very good at predicting strong companies and could identify these companies as being relatively undervalued, despite their true label not being too far off.

In [12]:
# find rows where label = 6 but model did not predict 6

false_negatives = results_df[(results_df['pred_label'] != 6) & (results_df['true_label'] == 6)]
print("False negatives:")
print(false_negatives)

False negatives:
             city state         zip        country  \
1036   Long Beach    CA       90831  United States   
712   Baton Rouge    LA       70808  United States   
1006  Sioux Falls    SD       57108  United States   
249     Greenwich    CT       06830  United States   
1088  Saint Louis    MO       63105  United States   
428      Stamford    CT       06902  United States   
633        Dallas    TX  75225-6068  United States   
621       Chicago    IL  60606-2609  United States   
80       San Jose    CA       95134  United States   
218      McKinney    TX  75070-8080  United States   
36        Houston    TX  77042-3643  United States   

                          industry                 industryKey  \
1036                 Oil & Gas E&P                 oil-gas-e-p   
712               REIT - Specialty              reit-specialty   
1006              Banks - Regional              banks-regional   
249                Capital Markets             capital-markets   
1088

Again we can see that the model performs quite well, only mislabeling 11 true labels for the 6 label. These stocks could be identified as being overvalued and could be used to open a short position.

In [13]:
# we now want to get the identifiers back for the companies that are mislabeled
false_positives_6 = df_raw.loc[false_positives.index, ['symbol', 'shortName']]
print('false positive companies')
print(false_positives_6)

false_negatives_6 = df_raw.loc[false_negatives.index, ['symbol', 'shortName']]
print('false negative companies:')
print(false_negatives_6)

false positive companies
     symbol                     shortName
720    LOPE  Grand Canyon Education, Inc.
1392     SM             SM Energy Company
false negative companies:
     symbol                        shortName
1036    CRC  California Resources Corporatio
712    LAMR        Lamar Advertising Company
1006   CASH         Pathward Financial, Inc.
249    IBKR  Interactive Brokers Group, Inc.
1088    ENR         Energizer Holdings, Inc.
428     SYF              Synchrony Financial
633     EXP              Eagle Materials Inc
621     ELS  Equity Lifestyle Properties, In
80     CDNS     Cadence Design Systems, Inc.
218      GL                  Globe Life Inc.
36      APA                  APA Corporation


These two lists can be used to identify companies that may be undervalued (false positives) or overvalued (false negatives)

Part 7 
Data inconsistency 1) - data leakage

In [ ]:
# use the same pipeline except this time leave returnOnEquity and
# operatingMargins in data leakage example
def data_leakage(df):
    df_clean = df.copy()
    # keep only rows with ROE values
    df_clean.dropna(subset=['returnOnEquity'], inplace=True)
    # split ROE and operating margins to quartiles, then get composite score
    df_clean['label'] = (
            (pd.qcut(df_clean['returnOnEquity'], q=4, labels=False)) +
            (pd.qcut(df_clean['operatingMargins'], q=4, labels=False))
    )
    # remove columns that have no value for analysis, have large amount of
    # missing data

    drop_cols = ['companyOfficers', 'website', 'phone', 'irWebsite',
                 'longBusinessSummary', 'address1', 'tradeable', 'quoteType',
                 'symbol', 'language', 'region', 'quoteSourceName',
                 'triggerable', 'customPriceAlertConfidence', 'marketState',
                 'exchangeDataDelayedBy', 'sourceInterval', 'cryptoTradeable',
                 'shortName', 'longName', 'hasPrePostMarketData',
                 'corporateActions', 'messageBoardId', 'exchangeTimezoneName',
                 'exchangeTimezoneShortName', 'gmtOffSetMilliseconds', 'fax',
                 'market', 'esgPopulated', 'address2', 'displayName',
                 'ipoExpectedDate', 'prevName', 'nameChangeDate',
                 'industrySymbol', 'prevTicker', 'tickerChangeDate',
                 'trailingPegRatio', 'lastSplitDate', 'lastSplitFactor']

    df_clean.drop(drop_cols, inplace=True, axis=1)
    return df_clean

# redefine df with leakage now
df_leaked = data_leakage(df_raw)

# train model with data leakage

# get all features, exclude label
features = df_leaked.drop(columns=['label'])
target = df_leaked['label']

# all other code is now reused, successfully added data leak
X_train, X_test, y_train, y_test = train_test_split(features,
                                                    target,
                                                    test_size=0.2,
                                                    random_state=44,
                                                    stratify=target)

# use median imputing for numerical values after test train split
# impute X train and X test
# get the numerical and categorical columns
num_features = features.select_dtypes(include=['int64', 'float64']).columns
cat_features = features.select_dtypes(include=['object']).columns

# define a pipeline for numerical imputing and min max scaling
num_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())])  # scale numerical features so no neg values


# define categorical feature pipeline with imputing and ordinal encoding
cat_pipeline_oe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(handle_unknown="use_encoded_value",
                               unknown_value=-1))
])


preprocessor_oe = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, num_features),
        ('cat', cat_pipeline_oe, cat_features)
    ],
    remainder='drop'  # ensures all columns are handled or dropped
)


grid_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor_oe),  # use the same preprocessor as before
    ('clf', DecisionTreeClassifier(random_state=44))  # use decision tree
])

# set the param grid that will be used in the pipeline
param_grid = {
    'clf__max_depth': [5, 6, 7, 8],  # max depth for tree
    'clf__min_samples_split': [2, 3, 4, 5],  # min samples split for tree
    'clf__min_samples_leaf': [6, 8, 10, 12, 13],  # min samples leaf for tree
    'clf__criterion': ['gini', 'entropy'] # split criterion
}

grid = GridSearchCV(
    grid_pipe,
    param_grid,
    cv=5,
    scoring='roc_auc_ovr_weighted',  # non-binary classification scoring
    n_jobs=-1
)

grid.fit(X_train, y_train)
print("Best params from grid search:", grid.best_params_)
print("Best ROC-AUC from grid search:", grid.best_score_)

y_pred_grid = grid.predict(X_test)
y_pred_proba_grid = grid.predict_proba(X_test)

Best params from grid search: {'clf__criterion': 'gini', 'clf__max_depth': 7, 'clf__min_samples_leaf': 8, 'clf__min_samples_split': 2}
Best ROC-AUC from grid search: 0.9950555611540878


Huge increase of ROC-AUC because of data leakage, showing the importance of removing all features that are used to derive the label

Part 7
2) do not remove any columns apart from return on equity and operating margins

In [ ]:
# use the same pipeline except this time leave all columns except returnOnEquity and
# operatingMargins in
def less_removal(df):
    df_clean = df.copy()
    # keep only rows with ROE values
    df_clean.dropna(subset=['returnOnEquity'], inplace=True)
    # split ROE and operating margins to quartiles, then get composite score
    df_clean['label'] = (
            (pd.qcut(df_clean['returnOnEquity'], q=4, labels=False)) +
            (pd.qcut(df_clean['operatingMargins'], q=4, labels=False))
    )
    # remove columns that have no value for analysis, have large amount of
    # missing data

    drop_cols = ['returnOnEquity', 'operatingMargins']

    df_clean.drop(drop_cols, inplace=True, axis=1)
    return df_clean

df_all = less_removal(df_raw)

# show that missing percentages have not changed since no columns removed
missing_pct = df_all.isnull().mean().sort_values(ascending=False)
print('Missing percentages after cleaning')
print(missing_pct.head(10))

# train model with all features except leak

# get all features, exclude label
features = df_all.drop(columns=['label'])
target = df_all['label']

X_train, X_test, y_train, y_test = train_test_split(features,
                                                    target,
                                                    test_size=0.2,
                                                    random_state=44,
                                                    stratify=target)

# use median imputing for numerical values after test train split
# impute X train and X test
# get the numerical and categorical columns
num_features = features.select_dtypes(include=['int64', 'float64']).columns
cat_features = features.select_dtypes(include=['object']).columns

# define a pipeline for numerical imputing and min max scaling
num_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())])  # scale numerical features so no neg values


# define categorical feature pipeline with imputing and ordinal encoding
cat_pipeline_oe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(handle_unknown="use_encoded_value",
                               unknown_value=-1))
])


preprocessor_oe = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, num_features),
        ('cat', cat_pipeline_oe, cat_features)
    ],
    remainder='drop'  # ensures all columns are handled or dropped
)


grid_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor_oe),  # use the same preprocessor as before
    ('clf', DecisionTreeClassifier(random_state=44))  # use decision tree
])

# set the param grid that will be used in the pipeline
param_grid = {
    'clf__max_depth': [5, 6, 7, 8],  # max depth for tree
    'clf__min_samples_split': [2, 3, 4, 5],  # min samples split for tree
    'clf__min_samples_leaf': [6, 8, 10, 12, 13],  # min samples leaf for tree
    'clf__criterion': ['gini', 'entropy'] # split criterion
}

grid = GridSearchCV(
    grid_pipe,
    param_grid,
    cv=5,
    scoring='roc_auc_ovr_weighted',  # non-binary classification scoring
    n_jobs=-1
)

grid.fit(X_train, y_train)
print("Best params from grid search:", grid.best_params_)
print("Best ROC-AUC from grid search:", grid.best_score_)

y_pred_grid = grid.predict(X_test)
y_pred_proba_grid = grid.predict_proba(X_test)

Missing percentages after cleaning
tickerChangeDate    1.000000
prevTicker          1.000000
industrySymbol      0.999293
ipoExpectedDate     0.960424
nameChangeDate      0.898940
prevName            0.898940
fax                 0.790813
trailingPegRatio    0.592933
address2            0.578092
irWebsite           0.485512
dtype: float64


/Users/brendantorok/Documents/Schooling/BU MET/CS767 - Advanced Machine Learning/Assignment 1/venv/lib/python3.11/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['industrySymbol' 'prevTicker' 'tickerChangeDate']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(
/Users/brendantorok/Documents/Schooling/BU MET/CS767 - Advanced Machine Learning/Assignment 1/venv/lib/python3.11/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['industrySymbol' 'prevTicker' 'tickerChangeDate']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(
/Users/brendantorok/Documents/Schooling/BU MET/CS767 - Advanced Machine Learning/Assignment 1/venv/lib/python3.11/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['industrySymbol' 'prevTicker' 'ticke

Best params from grid search: {'clf__criterion': 'entropy', 'clf__max_depth': 5, 'clf__min_samples_leaf': 12, 'clf__min_samples_split': 2}
Best ROC-AUC from grid search: 0.8171514331344332


/Users/brendantorok/Documents/Schooling/BU MET/CS767 - Advanced Machine Learning/Assignment 1/venv/lib/python3.11/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['industrySymbol' 'prevTicker' 'tickerChangeDate']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(
/Users/brendantorok/Documents/Schooling/BU MET/CS767 - Advanced Machine Learning/Assignment 1/venv/lib/python3.11/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['industrySymbol' 'prevTicker' 'tickerChangeDate']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(


Part 7 
3) all null numerical values changed to 0 instead of imputed

In [ ]:
# use the same pipeline except change impute technique for numerical values
def clean_data(df):
    df_clean = df.copy()
    # keep only rows with ROE values
    df_clean.dropna(subset=['returnOnEquity'], inplace=True)
    # split ROE and operating margins to quartiles, then get composite score
    df_clean['label'] = (
            (pd.qcut(df_clean['returnOnEquity'], q=4, labels=False)) +
            (pd.qcut(df_clean['operatingMargins'], q=4, labels=False))
    )
    # remove columns that have no value for analysis, have large amount of
    # missing data

    drop_cols = ['companyOfficers', 'website', 'phone', 'irWebsite',
                 'longBusinessSummary', 'address1', 'tradeable', 'quoteType',
                 'symbol', 'language', 'region', 'quoteSourceName',
                 'triggerable', 'customPriceAlertConfidence', 'marketState',
                 'exchangeDataDelayedBy', 'sourceInterval', 'cryptoTradeable',
                 'shortName', 'longName', 'hasPrePostMarketData',
                 'corporateActions', 'messageBoardId', 'exchangeTimezoneName',
                 'exchangeTimezoneShortName', 'gmtOffSetMilliseconds', 'fax',
                 'market', 'esgPopulated', 'address2', 'displayName',
                 'ipoExpectedDate', 'prevName', 'nameChangeDate',
                 'industrySymbol', 'prevTicker', 'tickerChangeDate',
                 'trailingPegRatio', 'lastSplitDate', 'lastSplitFactor',
                 'returnOnEquity', 'operatingMargins']

    df_clean.drop(drop_cols, inplace=True, axis=1)
    return df_clean

df_cleaned = clean_data(df_raw)

# train model to have constant imputing for numerical features

# get all features, exclude label
features = df_cleaned.drop(columns=['label'])
target = df_cleaned['label']

X_train, X_test, y_train, y_test = train_test_split(features,
                                                    target,
                                                    test_size=0.2,
                                                    random_state=44,
                                                    stratify=target)

# use median imputing for numerical values after test train split
# impute X train and X test
# get the numerical and categorical columns
num_features = features.select_dtypes(include=['int64', 'float64']).columns
cat_features = features.select_dtypes(include=['object']).columns

# define a pipeline for numerical imputing and min max scaling
num_pipeline = Pipeline(steps=[
    # simple change imputing strategty to be constant
    ('imputer', SimpleImputer(strategy='constant', fill_value=99999999999999)),
    ('scaler', MinMaxScaler())])  # scale numerical features so no neg values


# define categorical feature pipeline with imputing and ordinal encoding
cat_pipeline_oe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(handle_unknown="use_encoded_value",
                               unknown_value=-1))
])


preprocessor_oe = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, num_features),
        ('cat', cat_pipeline_oe, cat_features)
    ],
    remainder='drop'  # ensures all columns are handled or dropped
)

# use previous AI generated code to show impact on free cash flow 

# This section is to show the effects of the training for freeCashflow
# Fit the preprocessor on the training data
preprocessor_oe.fit(X_train)

# Transform both train and test sets
X_train_processed = preprocessor_oe.transform(X_train)
X_test_processed = preprocessor_oe.transform(X_test)

# Get the feature names (for numeric + encoded categorical features)
feature_names = preprocessor_oe.get_feature_names_out()

# Convert transformed data back into a DataFrame
X_train_df = pd.DataFrame(X_train_processed, columns=feature_names, index=X_train.index)
X_test_df = pd.DataFrame(X_test_processed, columns=feature_names, index=X_test.index)

# Concatenate them to get a full processed dataset for analysis
X_full_processed = pd.concat([X_train_df, X_test_df])

# Find the exact transformed column name
[col for col in X_full_processed.columns if "freeCashflow" in col]

fc_col = 'num__freeCashflow'

# Compute post-imputation + scaling stats
print(f"Median of {fc_col}:", X_full_processed[fc_col].median())
# add mean of freeCashflow to show that mean is skewed
print(f"Mean of {fc_col}:", X_full_processed[fc_col].mean())
print(f"Number of nulls in {fc_col}:", X_full_processed[fc_col].isnull().sum())
print(f"Range of {fc_col}: {X_full_processed[fc_col].max() - X_full_processed[fc_col].min()}")


grid_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor_oe),  # use the same preprocessor as before
    ('clf', DecisionTreeClassifier(random_state=44))  # use decision tree
])

# set the param grid that will be used in the pipeline
param_grid = {
    'clf__max_depth': [5, 6, 7, 8],  # max depth for tree
    'clf__min_samples_split': [2, 3, 4, 5],  # min samples split for tree
    'clf__min_samples_leaf': [6, 8, 10, 12, 13],  # min samples leaf for tree
    'clf__criterion': ['gini', 'entropy'] # split criterion
}

grid = GridSearchCV(
    grid_pipe,
    param_grid,
    cv=5,
    scoring='roc_auc_ovr_weighted',  # non-binary classification scoring
    n_jobs=-1
)

grid.fit(X_train, y_train)
print("Best params from grid search:", grid.best_params_)
print("Best ROC-AUC from grid search:", grid.best_score_)

y_pred_grid = grid.predict(X_test)
y_pred_proba_grid = grid.predict_proba(X_test)

Median of num__freeCashflow: 0.00010025353506716694
Mean of num__freeCashflow: 0.14567572886281666
Number of nulls in num__freeCashflow: 0
Range of num__freeCashflow: 1.0000400323877257
Best params from grid search: {'clf__criterion': 'gini', 'clf__max_depth': 5, 'clf__min_samples_leaf': 13, 'clf__min_samples_split': 2}
Best ROC-AUC from grid search: 0.8155389907160296


In [ ]:
# find percent of free cash flow empty
print(df_raw['freeCashflow'].isnull().sum())
print(df_raw['freeCashflow'].median())

243
276585936.0


The results of changing the imputation technique is that free cashflow has a left skew and the decision tree does not use it as a valuable feature anymore. The AUC-ROC drops slightly but is mostly unchanged because the model is robust.